<a href="https://colab.research.google.com/github/Churchiill/MilkywayFormation/blob/main/MilkyWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests, zipfile


# plt.rcParams.update({'font.size': 20, 'figsize':(8,6)})

SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rc('axes', linewidth = 5 )
plt.rc('lines', linewidth = 5 )

plt.rc('lines', markersize = 5 )

plt.rc('figure', figsize=(8,4) )         # Image size

# Import data

In [ ]:
url = 'https://mega.nz/file/54Ijzaob#OGxTt27c2IasKVwYy_yqfUffvdoyhrAKrqWhzKxs53s'
s=requests.get(url, stream=True).content
data = pd.read_csv(io.BytesIO(s) )



In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/drive/MyDrive/V1.rar').extractall('/content/drive/MyDrive')

In [ ]:
import csv

# read .dat to a list of lists
datContent = [i.strip().split() for i in open("/content/drive/MyDrive/OGLE-LMC-LPV-00001.dat").readlines()]

f = open("/content/drive/MyDrive/shitt.csv", "x")

# write it as a new CSV file
with open("/content/drive/MyDrive/shitt.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(datContent)



In [ ]:
import pandas as pd

dataframe = pd.read_csv("/content/drive/MyDrive/shitt.csv", names=['a','b','c'])

dataframe

,a,b,c
0,3032.76426,16.654,0.007
1,3326.85569,16.572,0.007
2,3337.70964,16.601,0.008
3,3353.67589,16.597,0.007
4,3358.67497,16.590,0.007
5,3366.67504,16.590,0.008
6,3375.64125,16.616,0.007
7,3385.71488,16.602,0.007
8,3397.63598,16.611,0.008
9,3720.68591,16.586,0.008
